In [1]:
from numba import njit, types
import h5py

import numpy as np, pandas as pd, scanpy as sc

In [2]:
# Functions for saving and loading clustering data
def write_results(pth, params, clusterings):
    with h5py.File(pth, "w") as f:
        cluster_group = f.create_group("clusterings")
        cluster_group.create_dataset(
            "clusterings", data=clusterings.values, compression="lzf"
        )
        cluster_group.create_dataset(
            "obs_names", data=pbmc.obs_names.values, compression="lzf"
        )

        params_group = f.create_group("params")
        for k, v in params.items():
            params_group.create_dataset(k, data=v.values, compression="lzf")


def read_results(pth) -> "Tuple[pd.DataFrame, pd.DataFrame]":
    """Read params and clusterings which have been stored to disk."""
    with h5py.File(pth, "r") as f:
        params_group = f["params"]
        params = pd.DataFrame(
            {
                col: params_group[col]
                for col in ["n_neighbors", "resolution", "random_state"]
            }
        )
        cluster_group = f["clusterings"]
        clusterings = pd.DataFrame(
            cluster_group["clusterings"][()],
            index=cluster_group["obs_names"].asstr()[:],
        )
    return params, clusterings

In [3]:
def relabel_clusterings(clusterings: pd.DataFrame) -> pd.DataFrame:
    """
    Make sure each cluster in a set of clusterings has a unique integer ID.
    
    Each clustering should have a contiguous range of integers for it's cluster's IDs.
    """
    clusterings = clusterings.rank(axis=0, method="dense").astype(int) - 1
    cvals = clusterings.values
    cvals[:, 1:] += (cvals[:, :-1].max(axis=0) + 1).cumsum()
    return clusterings

This notebook has two ways to compute the jaccard index between clusters in two clusterings. They are fundamentally the same, they are just different in how they store the intersection of clusters.

The first method stores these in an array of size `(number of clusters in clustering 1, number of clusters in clustering2)`. The second stores the intersections in a `dict`. Using the array is much faster when there is a small-ish number of clusters. The `dict` becomes faster when there is a large number of clusters.

Jaccard index is:

$$
\frac{| clustering1 \cap clustering2 |}{| clustering1 \cup clustering2 |}
$$

With the size of the union calculated as:

$$
| clustering1 \cup clustering2 | = |clustering1| + |clustering2| - | clustering1 \cap clustering2 |
$$

In [4]:
@njit
def clustering_edges_array(clustering1: "np.ndarray[int]", clustering2: "np.ndarray[int]") -> "list[tuple[int, int, float]]":
    """
    Find jaccard similarities between clusters in two clusterings.

    Params
    ------
    c1, c2
        Clusterings encoded as array of integers. Assumes each cluster has a unique integer id (will be node label in graph).

    Returns
    -------

    Edges in the cluster-cluster graph encoded as tuples of (node1, node2, jaccard similarity).
    """
    edges = []
    offset1 = clustering1.min()
    offset2 = clustering2.min()
    # Because of how I've done unique node names, potentially this
    # could be done in a more generic way by creating a mapping here.
    offset_clusts1 = clustering1 - offset1
    offset_clusts2 = clustering2 - offset2
    # Allocate coincidence matrix
    nclusts1 = offset_clusts1.max() + 1
    nclusts2 = offset_clusts2.max() + 1
    coincidence = np.zeros((nclusts1, nclusts2))
    # Allocate cluster size arrays
    ncells1 = np.zeros(nclusts1)
    ncells2 = np.zeros(nclusts2)
    # Compute lengths of the intersects
    for cell in range(len(clustering1)):
        c1 = offset_clusts1[cell]
        c2 = offset_clusts2[cell]
        # Coincidence matrix stores the size of the intersection of the clusters
        coincidence[c1, c2] += 1
        ncells1[c1] += 1
        ncells2[c2] += 1
    for cidx1, cidx2 in np.ndindex(coincidence.shape):
        isize = coincidence[cidx1, cidx2]
        if isize < 1:
            continue
        # Jaccard similarity is |intersection| / |union|.
        # Here, |union| is calculated as |clustering1| + |clustering2|
        jaccard_sim = isize / (ncells1[cidx1] + ncells2[cidx2] - isize)
        edge = (cidx1 + offset1, cidx2 + offset2, jaccard_sim)
        edges.append(edge)
    return edges

In [5]:
@njit
def clustering_edges_dict(clustering1: "np.ndarray[int]", clustering2: "np.ndarray[int]") -> "list[tuple[int, int, float]]":
    """
    Find jaccard similarities between clusters in two clusterings.

    Params
    ------
    c1, c2
        Clusterings encoded as array of integers. Assumes each cluster has a unique integer id (will be node label in graph).

    Returns
    -------

    Edges in the cluster-cluster graph encoded as tuples of (node1, node2, jaccard similarity).
    """
    edges = []
    # Because of how I've done unique node names, potentially these could be passed in to skip the computation
    offset1 = clustering1.min()
    offset2 = clustering2.min() 
    nclusts1 = clustering1.max() - offset1 + 1
    nclusts2 = clustering2.max() - offset2 + 1
    coincidence = dict()
    # Allocate cluster size arrays
    ncells1 = np.zeros(nclusts1)
    ncells2 = np.zeros(nclusts2)
    # Compute lengths of the intersects
    for cell in range(len(clustering1)):
        c1 = clustering1[cell]
        c2 = clustering2[cell]
        if (c1, c2) not in coincidence:
            coincidence[(c1, c2)] = 0
        coincidence[(c1, c2)] += 1
        ncells1[c1 - offset1] += 1
        ncells2[c2 - offset2] += 1
    for (cidx1, cidx2), isize in coincidence.items():
        jaccard_sim = isize / (ncells1[cidx1 - offset1] + ncells2[cidx2 - offset2] - isize)
        edge = (cidx1, cidx2, jaccard_sim)
        edges.append(edge)
    return edges

# Examples

## Using data from other tutorial

In [6]:
params, clusterings = read_results("../pbmc/data/pbmc-clusterings.h5")

In [7]:
clusterings = relabel_clusterings(clusterings)

In [8]:
# Number of clusters in each clustering
clusterings.apply(lambda x: len(x.unique()))

0          4
1          4
2          4
3          4
4          4
        ... 
1435     637
1436     819
1437    1054
1438    1326
1439    1656
Length: 1440, dtype: int64

### Performance

This could of course still be optimized. I have pretty much only used the array implementation because

1. When I wrote it, you couldn't use `dict`s in `numba`
2. It's fast enough to compute all the edges I needed. Analysis on the graph created from these edges takes much longer.

If we run into performance issues, we can optimize further (e.g. use `dict` implementation at high cluster number, pass offsets and number of cells per cluster, pre-allocate and reuse memory for coincidence matrix).

In [9]:
# JIT warmup
clustering_edges_array(clusterings.values[:, 0], clusterings.values[:, 1])
clustering_edges_dict(clusterings.values[:, 0], clusterings.values[:, 1]);

Array algorithm is fast for smaller number of clusters, but slow for smaller number of clusters

In [10]:
%timeit clustering_edges_array(clusterings.values[:, 0], clusterings.values[:, 1])
%timeit clustering_edges_array(clusterings.values[:, 1438], clusterings.values[:, 1439])

56.6 µs ± 10.6 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
4.63 ms ± 204 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Using the dictionary is slow for small number of clusters, but doesn't increase with larger numbers of clusters.

In [11]:
%timeit clustering_edges_dict(clusterings.values[:, 0], clusterings.values[:, 1])
%timeit clustering_edges_dict(clusterings.values[:, 1438], clusterings.values[:, 1439])

823 µs ± 170 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.16 ms ± 19.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Making sure they are equivalent

In [12]:
def check_same_result(c1, c2):
    results_array = sorted(clustering_edges_array(c1, c2))
    results_dict = sorted(clustering_edges_dict(c1, c2))
    return results_array == results_dict

In [13]:
assert check_same_result(clusterings.values[:, 0], clusterings.values[:, 1])
assert check_same_result(clusterings.values[:, 1438], clusterings.values[:, 1439])

## Self contained example

Here's a small example showing how to generate clusterings and compare them.

In [14]:
import scanpy as sc

pbmc = sc.datasets.pbmc3k_processed().raw.to_adata()
sc.pp.pca(pbmc)
sc.pp.neighbors(pbmc)

clustering_keys = []
for res in [0.01, 0.02, 50., 100.]:
    key = f"leiden-res-{res}"
    sc.tl.leiden(pbmc, resolution=res, key_added=key)
    clustering_keys.append(key)

OMP: Info #271: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [15]:
clustering_keys

['leiden-res-0.01', 'leiden-res-0.02', 'leiden-res-50.0', 'leiden-res-100.0']

In [16]:
pbmc.obs[clustering_keys] = relabel_clusterings(pbmc.obs[clustering_keys])

In [17]:
clusterings = pbmc.obs[clustering_keys]

In [18]:
clustering_edges_array(clusterings["leiden-res-0.01"].values, clusterings["leiden-res-0.02"].values)

[(0, 3, 0.8249744114636642),
 (0, 5, 0.1750255885363357),
 (1, 4, 1.0),
 (2, 6, 1.0)]

In [19]:
clustering_edges_array(clusterings["leiden-res-50.0"].values, clusterings["leiden-res-100.0"].values)

[(7, 420, 0.6666666666666666),
 (7, 829, 0.05),
 (7, 1122, 0.2777777777777778),
 (8, 454, 0.5333333333333333),
 (8, 499, 0.3333333333333333),
 (8, 793, 0.058823529411764705),
 (8, 987, 0.05),
 (9, 442, 0.38461538461538464),
 (9, 756, 0.06666666666666667),
 (9, 809, 0.1875),
 (9, 927, 0.15384615384615385),
 (9, 992, 0.15384615384615385),
 (10, 484, 0.4444444444444444),
 (10, 667, 0.5),
 (11, 425, 0.625),
 (11, 472, 0.3),
 (12, 430, 0.625),
 (12, 766, 0.2222222222222222),
 (12, 836, 0.1),
 (13, 643, 0.875),
 (13, 682, 0.1),
 (14, 890, 0.375),
 (14, 964, 0.5555555555555556),
 (15, 428, 0.625),
 (15, 702, 0.375),
 (16, 603, 0.5),
 (16, 1126, 0.4444444444444444),
 (17, 493, 0.625),
 (17, 736, 0.375),
 (18, 462, 0.625),
 (18, 1128, 0.3),
 (19, 471, 0.08333333333333333),
 (19, 742, 0.875),
 (20, 508, 0.2857142857142857),
 (20, 530, 0.3076923076923077),
 (20, 720, 0.17647058823529413),
 (20, 1107, 0.11764705882352941),
 (21, 654, 0.875),
 (21, 902, 0.1),
 (22, 479, 0.625),
 (22, 800, 0.1),
 (2